<a href="https://colab.research.google.com/github/ayuksekdag/pygrametl/blob/main/pygrametl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



*https://chrthomsen.github.io/pygrametl/doc/quickstart/beginner.html




In [2]:
!pip install pygrametl
#!pip install sqlite3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [38]:
import sqlite3
import pygrametl
from pygrametl.tables import Dimension

# Input is a list of rows which in pygrametl is modeled as dicts
products = [{
    'name': 'Calvin and Hobbes 1',
    'category': 'Comic',
    'price': '10'
}, {
    'name': 'Calvin and Hobbes 2',
    'category': 'Comic',
    'price': '10'
}, {
    'name': 'Calvin and Hobbes 3',
    'category': 'Comic',
    'price': '10'
}, {
    'name': 'Cake and Me',
    'category': 'Cookbook',
    'price': '15'
}, {
    'name': 'French Cooking',
    'category': 'Cookbook',
    'price': '50'
}, {
    'name': 'Sushi',
    'category': 'Cookbook',
    'price': '30'
}, {
    'name': 'Nineteen Eighty-Four',
    'category': 'Novel',
    'price': '15'
}, {
    'name': 'The Lord of the Rings',
    'category': 'Novel',
    'price': '60'
}]

# The actual database connection is handled by a PEP 249 connection

pgconn = sqlite3.connect(":memory:", detect_types=sqlite3.PARSE_DECLTYPES)

# This ContectionWrapper will be set as a default and is then implicitly
# used, but it is stored in conn so transactions can be committed and the
# connection closed
conn = pygrametl.ConnectionWrapper(connection=pgconn)
cur = conn.cursor()

cur.execute(
    'create table product (product_id int, name string, category string, price int)'
)

# The instance of Dimension connects to the table product in the
# database using the default connection wrapper created above, the
# argument lookupatts specifies the column which needs to match
# when doing a lookup for the key from this dimension
productDimension = Dimension(name='product',
                             key='product_id',
                             attributes=['name', 'category', 'price'],
                             lookupatts=['name'])

# Filling a dimension is simply done by using the insert method

cur.execute('delete from product')

for row in products:
    productDimension.insert(row)

# Ensures that the data is committed and the connection is closed correctly
conn.commit()
#conn.close()

In [39]:
import pandas as pd

df = pd.read_sql_query("select product_id, name, category, price from product", conn)
df.head()

,product_id,name,category,price


In [40]:
df2 = df.groupby(['category'])['price'].sum()

In [41]:
df2.plot(kind='bar')

TypeError: ignored